In [1]:
# Inspired by https://www.sciencedirect.com/science/article/pii/S0377221723000735
# https://www.sciencedirect.com/science/article/pii/S0167637704000537
# https://medium.com/swlh/techniques-for-subtour-elimination-in-traveling-salesman-problem-theory-and-implementation-in-71942e0baf0c

In [2]:
import gurobipy as gp

# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [3]:
# Initialize problem parameters
scraps = 5 #32
max_boxes = 6 #39
max_points = max_boxes * 2 + 2

set_scraps = range(0, scraps)
set_boxes = range(0, max_boxes)
set_points = range(0, max_points)

# Minium of 3 scraps per recipe, MUST USE ALL SCRAPS IN AT LEAST ONE RECIPE
recipes = [[0, 1, 2, 3], [1, 2, 3], [3, 2, 4], [4, 3, 2]]
                                 
set_recipes = range(0, len(recipes))

In [4]:
# Create variables

# Scrap s is assigned to box b
x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

# Scrap s is assigned to point w
a = {}
for scrap in set_scraps:
    for point in set_points[1:]:
        a[scrap, point] = m.addVar(vtype='B', name=f"a[{scrap},{point}]")

# Box b is in solution path for recipe r
y = {}
for box in set_boxes:
    for recipe in set_recipes:
        y[box, recipe] = m.addVar(vtype='B', name=f"y[{box},{recipe}]")

# Box b is in the global solution
g = {}
for box in set_boxes:
    g[box] = m.addVar(vtype='B', name=f"g[{box}]")

# Distance between start and point w
w = {}
for w1 in set_points:
    w[w1] = m.addVar(vtype='C', name=f"w[{w1}]")

# Distance between two points w1 and w2
d = {}
for w1 in set_points:
    for w2 in set_points:
        if w1 == w2: continue
        d[w1, w2] = m.addVar(vtype='C', name=f"d[{w1},{w2}]")

# Length of box b
l = {}
for box in set_boxes:
    l[box] = m.addVar(vtype='C', name=f"l[{box}]")

# Point w is in solution path for recipe r
p = {}
for w1 in set_points:
    for recipe in set_recipes:
        p[w1, recipe] = m.addVar(vtype='B', name=f"p[{w1},{recipe}]")

# Edge w1 -> w2 is in solution path for recipe r
z = {}
for w1 in set_points:
    for w2 in set_points:
        if w1 != w2:
            for recipe in set_recipes:
                z[w1, w2, recipe] = m.addVar(vtype='B', name=f"z[{w1},{w2},{recipe}]")


In [5]:
# Add constraints

# Each scrap is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes) >= 1)

# point 0 is in solution path for all recipes
for r in set_recipes:
    m.addConstr(p[0, r] == 1)

# Each box is assigned at most one scrap
for b in set_boxes:
    m.addConstr(sum(x[s, b] for s in set_scraps) <= g[b])

# The distance to point 1 is 100m
m.addConstr(w[1] == 100)
m.addConstr(w[0] == 0)

# Points p1 and p2 of box b are assigned to the same scrap
for b in set_boxes:
    for s in set_scraps:
        m.addConstr(x[s, b] == a[s, 2*b + 1])
        m.addConstr(x[s, b] == a[s, 2*b + 2])

# The number of boxes in a solution for recipe r is equal to the number of scraps required by the recipe
for r in set_recipes:
    m.addConstr(sum(y[b, r] for b in set_boxes) == len(recipes[r]))

# There is exactly one box b in the solution path for recipe r that corresponds to each scrap s
for r in set_recipes:
    for s in recipes[r]:
        m.addConstr(sum(x[s, b] * y[b, r] for b in set_boxes) == 1)

# There is one incoming edge to each point w in the solution path for recipe r
for r in set_recipes:
    for w2 in set_points:
        m.addConstr(sum(z[w1, w2, r] for w1 in set_points if w1 != w2) == p[w2, r])

# There is one outgoing edge from each point w in the solution path for recipe r
for r in set_recipes:
    for w1 in set_points:
        m.addConstr(sum(z[w1, w2, r] for w2 in set_points  if w1 != w2) == p[w1, r])

# The distance between edge points of a box (in the solution) is larger than 10m
for b in set_boxes:
    m.addConstr(w[2*b + 2] - w[2*b + 1] >= 10 * g[b])
    # m.addGenConstrIndicator(g[b], True, w[2*b + 2] - w[2*b + 1] >= 10)

# The distance between edge points of a wall (in the solution) is 1.7m
for b in set_boxes:
    m.addConstr(w[2*b + 3] - w[2*b + 2] == 1.7 * g[b])
    # m.addGenConstrIndicator(g[b], True, w[2*b + 3] - w[2*b + 2] == 1.7)

# Calculate the distances between all points
for w1 in set_points:
    for w2 in set_points:
        if w2 < w1 or w1 == w2: continue
        m.addConstr(d[w1, w2] == w[w2] - w[w1])
        m.addConstr(d[w2, w1] == d[w1, w2])

# The length of a box (in the solution) is the distance between the first and last point of the box
for b in set_boxes:
    m.addConstr(l[b] == (w[2*b + 2] - w[2*b + 1]) * g[b])
    # m.addGenConstrIndicator(g[b], True, l[b] == w[2*b + 2] - w[2*b + 1])

# Box b is in the global solution if it is in the solution path for any recipe
for b in set_boxes:
    # m.addConstr(g[b] == sum(y[b, r] for r in set_recipes)) #Broken??
    m.addGenConstrIndicator(g[b], True, sum(y[b, r] for r in set_recipes) >= 1)

# If a box b is not in the global solution, all later boxes are also not in the global solution
for b1 in set_boxes:
    m.addConstr(sum(g[b2] for b2 in set_boxes if b2 > b1) <= g[b1] * max_boxes)

# If a box is in the solution path for recipe r, then one of the edge points of the box is also in the solution path for recipe r
for r in set_recipes:
    for b in set_boxes:
        m.addConstr(y[b, r] == p[2*b + 1, r] + p[2*b + 2, r])

# Only one edge of a box can be in the solution path for recipe r
for r in set_recipes:
    for b in set_boxes:
        m.addConstr(p[2*b + 1, r] + p[2*b + 2, r] <= 1)

# Ordering constraints
for r in set_recipes:
    for w1 in set_points[1:]:
        for w2 in set_points[1:]:
            if w1 == w2: continue
            m.addConstr(z[w1, w2, r] <= sum(a[recipes[r][i], w1] * a[recipes[r][i+1], w2] for i in range(len(recipes[r])-1)))
            
        

In [6]:
# Set objective function

objective = gp.quicksum(d[w1, w2] * z[w1, w2, r] for w1 in set_points for w2 in set_points for r in set_recipes if w1 != w2)
# objective = gp.quicksum(l[b] for b in set_boxes)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [7]:
# Solve it!
m.Params.timeLimit = 60.0
m.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 1111 columns and 2416 nonzeros
Model fingerprint: 0x4118eb2b
Model has 728 quadratic objective terms
Model has 643 quadratic constraints
Model has 6 general constraints
Variable types: 202 continuous, 909 integer (909 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
  QRHS range       [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 18 rows and 0 columns
Presolve removed 0 rows and 12 columns
Presol

In [8]:
import math
# Show solution
print(f"Optimal objective value: {m.objVal}")

# Print scrap assignments
scrap_assignments = {}
for box in set_boxes:
    for scrap in set_scraps:
        if x[scrap, box].X > 0:
            scrap_assignments[box] = scrap
            print(f"Scrap {scrap} is assigned to box {box}")

# Print box lengths
for box in set_boxes:
    if l[box].X > 0:
        print(f"Box {box} length: {l[box].X}")

# Print part of point distances
for w1 in set_points[:-1]:
    w2 = w1 + 1
    if d[w1, w2].X > 0:
        print(f"Distance between point {w1} and point {w2}: {d[w1, w2].X}")

# Print solution paths
def get_box_index(point):
    return math.floor((point - 1) // 2)

for recipe in set_recipes:
    print(f"Recipe {recipe} solution path:")
    for w1 in set_points:
        for w2 in set_points:
            if w1 != w2 and z[w1, w2, recipe].X > 0:
                if w1 == 0:
                    print(f"Start -> Box {get_box_index(w2)}: Scrap: {scrap_assignments[get_box_index(w2)]}")
                elif w2 == 0:
                    print(f"Box {get_box_index(w1)}: Scrap: {scrap_assignments[get_box_index(w1)]} -> Start")
                else:
                    print(f"Box {get_box_index(w1)}: Scrap: {scrap_assignments[get_box_index(w1)]} -> Box {get_box_index(w2)}: Scrap: {scrap_assignments[get_box_index(w2)]}")
    

Optimal objective value: 1057.4
Scrap 4 is assigned to box 0
Scrap 3 is assigned to box 1
Scrap 2 is assigned to box 2
Scrap 1 is assigned to box 3
Scrap 0 is assigned to box 4
Box 0 length: 10.0
Box 1 length: 10.0
Box 2 length: 10.0
Box 3 length: 10.0
Box 4 length: 10.0
Distance between point 0 and point 1: 100.0
Distance between point 1 and point 2: 10.0
Distance between point 2 and point 3: 1.7
Distance between point 3 and point 4: 10.0
Distance between point 4 and point 5: 1.7
Distance between point 5 and point 6: 10.0
Distance between point 6 and point 7: 1.7
Distance between point 7 and point 8: 10.0
Distance between point 8 and point 9: 1.7
Distance between point 9 and point 10: 10.0
Distance between point 10 and point 11: 1.7
Recipe 0 solution path:
Start -> Box 4: Scrap: 0
Box 1: Scrap: 3 -> Start
Box 2: Scrap: 2 -> Box 1: Scrap: 3
Box 3: Scrap: 1 -> Box 2: Scrap: 2
Box 4: Scrap: 0 -> Box 3: Scrap: 1
Recipe 1 solution path:
Start -> Box 3: Scrap: 1
Box 1: Scrap: 3 -> Start
Box